In [2]:
import pickle as pkl
import numpy as np
from tqdm import tqdm
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments, pipeline
from nltk.tokenize import sent_tokenize, word_tokenize
import multiprocessing
import nltk
nltk.download('punkt')
import evaluate


[2023-12-02 03:01:12,164] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


[nltk_data] Downloading package punkt to /home/haskari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('data_paraphrase/cnn.pkl', 'rb') as f:
    cnn_chat=pkl.load(f)

In [3]:
len(cnn_chat)

11490

In [4]:
cnn_chat[13]

[]

In [5]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(cnn_chat):
    if item == []:
        count+=1
        bad_index.append(idx)
print(count)



543


In [6]:
count=0
for items in cnn_chat:
    # items=nltk.sent_tokenize(items)

    if not items:
        continue
    elif len(items)==3:
        count+=1

print(count)

10939


In [7]:
print(len(cnn_chat))

11490


In [8]:
data = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'article'
summary_key = 'highlights'

data=data['test']

In [9]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

Parameter 'indices'=<generator object <genexpr> at 0x7f2996840900> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [10]:
def tokenize(example):

    example["original_article"] = example[article_key]
    example["article"] = nltk.sent_tokenize(example[article_key])
    # if args.dataset == 'news':
    # check=example[summary_key][0]['text']
    # example["highlights"] = nltk.sent_tokenize(check)
    # else:
    example["highlights"] = nltk.sent_tokenize(example[summary_key])
        
    return example

data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

Map (num_proc=48):   0%|          | 0/10947 [00:00<?, ? examples/s]

In [11]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

41255


In [12]:
count=0
for summ in cnn_chat:
    count+=len(summ)

print(count)
    

32827


In [13]:
with open('data_paraphrase/xsum.pkl', 'rb') as f:
    xsum_chat=pkl.load(f)

In [14]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(xsum_chat):
    if item == []:
        count+=1
        bad_index.append(idx)
print(count)


95


In [15]:
count=0
for items in xsum_chat:
    #items=nltk.sent_tokenize(items)
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

11237


In [16]:
data = load_dataset("xsum")
article_key = 'document'
summary_key = 'summary'

data=data['test']

In [17]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

In [18]:
data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

Map (num_proc=48):   0%|          | 0/11239 [00:00<?, ? examples/s]

In [19]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

11240


In [20]:
count=0
for summ in xsum_chat:
    count+=len(summ)

print(count)

11243


In [3]:
with open('data_paraphrase/news.pkl', 'rb') as f:
    news_chat=pkl.load(f)

In [4]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(news_chat):
    if item == []:
        count+=1
        bad_index.append(idx)
print(count)


7


In [5]:
count=0
for items in news_chat:
    items=items
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

992


In [6]:
data = load_dataset("argilla/news-summary")
article_key = 'text'
summary_key = 'prediction'
data = DatasetDict({
    'train': data['test'],
    'test': data['train']})

data=data['test']

In [7]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

Parameter 'indices'=<generator object <genexpr> at 0x7fc3b65291c0> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [8]:
def tokenize_news(example):
    example["original_article"] = example[article_key]
    example["article"] = nltk.sent_tokenize(example[article_key])

    check=example[summary_key][0]['text']
    example["highlights"] = nltk.sent_tokenize(check)

    return example

In [9]:
data = data.map(tokenize_news, num_proc=multiprocessing.cpu_count())

Map (num_proc=48):   0%|          | 0/993 [00:00<?, ? examples/s]

In [10]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

1005


In [11]:
count=0
for summ in news_chat:
    count+=len(summ)

print(count)

994


In [30]:
with open('data_paraphrase/reddit.pkl', 'rb') as f:
    reddit_chat=pkl.load(f)

In [31]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(reddit_chat):
    if item == []:
        count+=1
        bad_index.append(idx)
print(count)

757


In [32]:
count=0
for items in reddit_chat:
    #items=nltk.sent_tokenize(items)
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

3457


In [33]:
data = load_dataset('reddit_tifu', 'long')
article_key = 'documents'
summary_key = 'tldr'
    # 80% train, 20% test + validation
train_testvalid = data['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
data = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

data=data['test']

In [34]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

In [35]:
data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

Map (num_proc=48):   0%|          | 0/3457 [00:00<?, ? examples/s]

In [36]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

4789


In [37]:
count=0
for summ in reddit_chat:
    count+=len(summ)

print(count)

3457
